In [1]:
import time

import GPRutils
import vK2KGPR
import DESutils

import numpy as np
import astropy.table as tb
import astropy.units as u

import warnings
warnings.filterwarnings("ignore")

exps = DESutils.findExpNums()
gband = DESutils.bandDict["g"]
rband = DESutils.bandDict["r"]
iband = DESutils.bandDict["i"]
zband = DESutils.bandDict["z"]

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [4]:
def fixedKernelComparison(dC1, dC2, name1, name2, JK=True, printing=True):
    """
    Take final parameters from dC1 and fit them with the data from dC2.
    """
    GP = vK2KGPR.vonKarman2KernelGPR(dC2, curl=dC2.curl)
    if JK:
        dC2.JackKnife(GP, dC1.params, nomask=True)    
        
        xi0, xif = dC2.JackKnifeXi()
        xi0, Xerr, Yerr = xi0[0].value, xi0[1].value, xi0[2].value
        xi0err = np.sqrt(Xerr**2 + Yerr**2)
        xif, Xerr, Yerr = xif[0].value, xif[1].value, xif[2].value
        xiferr = np.sqrt(Xerr**2 + Yerr**2)
        red = xi0/xif
        rederr = np.sqrt(((xi0err/xi0)**2 + (xiferr/xif)**2) * red**2)
        
    else:
        GP.fit(dC1.params)
        GP.predict(dC2.Xvalid)
        
        xi0, Xerr, Yerr, prs = GPRutils.getXi(dC2.Xvalid, dC2.Yvalid)
        xi0err = np.sqrt(Xerr**2 + Yerr**2)
        xif, Xerr, Yerr, prs = GPRutils.getXi(dC2.Xvalid, dC2.Yvalid-dC2.fbar_s)
        xiferr = np.sqrt(Xerr**2 + Yerr**2)
        red = xi0/xif
        rederr = np.sqrt(((xi0err/xi0)**2 + (xiferr/xif)**2) * red**2)
    
    N = len(dC2.TV[dC2.TV["Maskf"]])
    starDensity = (N / (3*u.deg**2)).to(u.arcmin**-2).value
    
    if printing:
        line = [name1, name2, f"{xi0:.2f} ± {xi0err:.2f}", f"{xif:.2f} ± {xiferr:.2f}", f"{red:.2f} ± {rederr:.2f}", f"{starDensity:.2f}"]
        line.extend(np.abs(np.around(dC1.params, 7)))
        line = "".join([f"{param:<15}" for param in line])
        print(line)

    return xi0, xi0err, xif, xiferr, red, rederr, starDensity

In [6]:
for expNum in zband[:5] + gband[:5]:
    R = GPRutils.loadFITS(f"/home/fortino/GPRSolutions/NelderMead/GPR.{expNum}.?R.fits")
    RC = GPRutils.loadFITS(f"/home/fortino/GPRSolutions/NelderMead/GPR.{expNum}.?RC.fits")
    R.summarize(noplots=True)
    RC.summarize(noplots=True)
    fixedKernelComparison(R, RC, "R", "RC", JK=True, printing=True)
    print()
    print()

#####------348819 z------#####
Fitted von Kármán kernel parameters:
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
109.1565559    1.5349095      -0.0061141     0.0890528      -0.0355237     
xi0: 83.084 ± 1.757
xif: 11.122 ± 0.694
Reduction: 7.470 ± 0.492

Final von Kármán kernel parameters:
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
197.9888305    1.0227235      -0.0107593     0.0089676      -0.0435881     
xi0: 83.084 ± 1.757
xif: 6.606 ± 0.606
Reduction: 12.578 ± 1.185

#####------348819 z------#####
Fitted von Kármán kernel parameters:
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
109.1565559    1.5349095      -0.0061141     0.0890528      -0.0355237     

Final von Kármán kernel parameters:
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
220.7504306    0.5817545      -0.0084507     0.0122671      -0.030748      
xi0: 83.083 ± 1.693
xif: 5.641 ± 0.486
Reduction: 1

In [7]:
for expNum in zband[:5] + gband[:5]:
    R = GPRutils.loadFITS(f"/home/fortino/GPRSolutions/NelderMead_max/GPR.{expNum}.?R.fits")
    RC = GPRutils.loadFITS(f"/home/fortino/GPRSolutions/NelderMead_max/GPR.{expNum}.?RC.fits")
    R.summarize(noplots=True)
    RC.summarize(noplots=True)
    fixedKernelComparison(R, RC, "R", "RC", JK=True, printing=True)
    print()
    print()

#####------348819 z------#####
Fitted von Kármán kernel parameters:
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
82.1578936     1.1654622      0.0376682      0.1202223      -0.0545815     

Final von Kármán kernel parameters:
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
429.2719121    1.6826592      0.0134032      0.0016738      -0.0733438     
xi0: 82.824 ± 1.718
xif: 5.856 ± 0.524
Reduction: 14.143 ± 1.300

#####------348819 z------#####
Fitted von Kármán kernel parameters:
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
82.1578936     1.1654622      0.0376682      0.1202223      -0.0545815     

Final von Kármán kernel parameters:
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
324.3305769    1.533597       0.0287053      -0.0171639     -0.0306427     
xi0: 82.979 ± 1.719
xif: 4.991 ± 0.502
Reduction: 16.627 ± 1.707

R              RC             82.98 ± 1.72   5.22 

In [5]:
JK = True
printing = True
for expNum in zband[:5]:
    dCs = {
        "R": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/simple/GPR.{expNum}.?R.fits"),
        "Rmin": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/varClipping_L2/GPR.{expNum}.?R.fits"),
        "Rmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/varClipping_G250/GPR.{expNum}.?R.fits"),
        "Rminmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/varClipping_L2_G250/GPR.{expNum}.?R.fits"),

        "RC": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/simple/GPR.{expNum}.?RC.fits"),
        "RCmin": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/varClipping_L2/GPR.{expNum}.?RC.fits"),        
        "RCmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/varClipping_G250/GPR.{expNum}.?RC.fits"),
        "RCminmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/varClipping_L2_G250/GPR.{expNum}.?RC.fits"),
    }
    
    name2 = "R"
    dCs[name2].summarize(noplots=True)
    header = ["Params From", "Data From", "xi0", "xif", "Reduction", "Stars/arcmin^2", "K Variance", "Outer Scale", "Diameter", "Wind X", "Wind Y"]
    header = "".join([f"{name:<15}" for name in header])
    print(header)

    name1 = "Rmin"
    xi = fixedKernelComparison(dCs[name1], dCs[name2], name1, name2, JK=JK, printing=printing)
    
    name1 = "Rmax"
    xi = fixedKernelComparison(dCs[name1], dCs[name2], name1, name2, JK=JK, printing=printing)
    
    name1 = "Rminmax"
    xi = fixedKernelComparison(dCs[name1], dCs[name2], name1, name2, JK=JK, printing=printing)
    
    
    print()
    name2 = "RC"
    dCs[name2].summarize(noplots=True)    
    print(header)

    name1 = "RCmin"
    xi = fixedKernelComparison(dCs[name1], dCs[name2], name1, name2, JK=JK, printing=printing)
    
    name1 = "RCmax"
    xi = fixedKernelComparison(dCs[name1], dCs[name2], name1, name2, JK=JK, printing=printing)
    
    name1 = "RCminmax"
    xi = fixedKernelComparison(dCs[name1], dCs[name2], name1, name2, JK=JK, printing=printing)
    print()
    print()

348819, z
Optimzation    K Variance     Outer Scale    Diameter       Wind X         Wind Y         
fitCorr        109.1565559    1.5349095      -0.0061141     0.0890528      -0.0355237     
final          394.0468822    0.6892833      -0.0164337     0.0033909      -0.0150839     
xi0: 83.034 ± 1.687
xif: 6.916 ± 0.514
Reduction: 12.007 ± 0.926

Params From    Data From      xi0            xif            Reduction      Stars/arcmin^2 K Variance     Outer Scale    Diameter       Wind X         Wind Y         
Rmin           R              83.03 ± 1.69   7.25 ± 0.52    11.46 ± 0.85   0.91           691.8209319    0.1622388      0.2892273      0.0063522      0.0260261      
Rmax           R              83.03 ± 1.69   6.53 ± 0.51    12.71 ± 1.02   0.91           429.2719121    1.6826592      0.0134032      0.0016738      0.0733438      
Rminmax        R              83.03 ± 1.69   6.50 ± 0.50    12.77 ± 1.01   0.91           308.9158892    0.6462697      0.0123495      0.0182522      0.0

IndexError: list index out of range

In [ ]:
JK = True
printing = True
for expNum in gband[:5]:
    dCs = {
        "R": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/simple/GPR.{expNum}.?R.fits"),
        "Rmin": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/varClipping_L2/GPR.{expNum}.?R.fits"),
        "Rmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/varClipping_G250/GPR.{expNum}.?R.fits"),
        "Rminmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/varClipping_L2_G250/GPR.{expNum}.?R.fits"),

        "RC": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/simple/GPR.{expNum}.?RC.fits"),
        "RCmin": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/varClipping_L2/GPR.{expNum}.?RC.fits"),        
        "RCmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/varClipping_G250/GPR.{expNum}.?RC.fits"),
        "RCminmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/varClipping_L2_G250/GPR.{expNum}.?RC.fits"),
    }
    
    name2 = "R"
    dCs[name2].summarize(noplots=True)
    header = ["Params From", "Data From", "xi0", "xif", "Reduction", "Stars/arcmin^2", "K Variance", "Outer Scale", "Diameter", "Wind X", "Wind Y"]
    header = "".join([f"{name:<15}" for name in header])
    print(header)

    name1 = "Rmin"
    xi = fixedKernelComparison(dCs[name1], dCs[name2], name1, name2, JK=JK, printing=printing)
    
    name1 = "Rmax"
    xi = fixedKernelComparison(dCs[name1], dCs[name2], name1, name2, JK=JK, printing=printing)
    
    name1 = "Rminmax"
    xi = fixedKernelComparison(dCs[name1], dCs[name2], name1, name2, JK=JK, printing=printing)
    
    
    print()
    name2 = "RC"
    dCs[name2].summarize(noplots=True)    
    print(header)

    name1 = "RCmin"
    xi = fixedKernelComparison(dCs[name1], dCs[name2], name1, name2, JK=JK, printing=printing)
    
    name1 = "RCmax"
    xi = fixedKernelComparison(dCs[name1], dCs[name2], name1, name2, JK=JK, printing=printing)
    
    name1 = "RCminmax"
    xi = fixedKernelComparison(dCs[name1], dCs[name2], name1, name2, JK=JK, printing=printing)
    print()
    print()